In [1]:
import math
import cooler
import numpy as np
import pandas as pd
import h5py
import dask
import dask.array as da
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
import zarr

In [2]:
###### Idea 1: Read in pixel table as hdf5 and convert to dask dataframe
cool = 's38_221203_Granta519EBF1KI_cl27_0hr_MicroC_25U_Nova.mcool'
res = 5000
f = h5py.File(cool, 'a')
bin1 = da.from_array(f['/resolutions/{}/pixels/bin1_id'.format(res)], chunks=(3000000,))
bin2 = da.from_array(f['/resolutions/{}/pixels/bin2_id'.format(res)], chunks=(3000000,))
count = da.from_array(f['/resolutions/{}/pixels/count'.format(res)], chunks=(3000000,))
df = dd.concat([dd.from_dask_array(d) for d in [bin1, bin2, count]], axis=1)
cols = ['bin1','bin2','counts']
df_bins = df.rename(columns=dict(zip(df.columns, cols)))
df_bins

,bin1,bin2,counts
npartitions=70,,,
0,int64,int64,int32
3000000,...,...,...
...,...,...,...
207000000,...,...,...
207915424,...,...,...


In [3]:
def correct_upper_triangle(x):
    mask = x['bin1'].values != x['bin2'].values
    x.loc[mask, 'counts'] *= 2
    return x
with ProgressBar():
    corrected_pixels = df_bins.map_partitions(correct_upper_triangle).compute()

[########################################] | 100% Completed | 29.33 s


In [4]:
# Sum rows
with ProgressBar():
    corrected_rowSums = dd.from_pandas(corrected_pixels, chunksize=3000000).groupby("bin1").counts.sum().compute()
corrected_rowSums

[########################################] | 100% Completed | 1.90 ss


bin1
2          54
3           4
10         42
11         43
12         20
         ... 
619140     23
627469    413
627470    142
627471    305
627472    111
Name: counts, Length: 561450, dtype: int32

In [5]:
# Store full matrix size
mat_size = da.from_array(f['/resolutions/{}/bins/chrom'.format(res)]).size

In [6]:
# Reindex and fill with 0
corrected_rowSums = corrected_rowSums.reindex(range(0,mat_size), fill_value=0)
corrected_rowSums

bin1
0          0
1          0
2         54
3          4
4          0
          ..
627473     0
627474     0
627475     0
627476     0
627477     0
Name: counts, Length: 627478, dtype: int32

In [9]:
if len(corrected_rowSums) == mat_size: print('Rowsums and matrix sizes match') # Sanity check for making sure row count length = matrix size)
else:     print('No matchy')

Rowsums and matrix sizes match


In [12]:
corrected_pixels

,bin1,bin2,counts
0,2,50195,2
1,2,103475,2
2,2,108205,2
3,2,127201,2
4,2,127970,2
...,...,...,...
207915420,627470,627471,86
207915421,627470,627472,36
207915422,627471,627471,207
207915423,627471,627472,98


In [22]:
i = corrected_pixels['bin1'].values
j = corrected_pixels['bin2'].values

In [27]:
rf = corrected_rowSums[i].values
cf = corrected_rowSums[j].values

In [30]:
scaled_pixels = corrected_pixels['counts'] / (rf*cf)

In [33]:
scaled_pixels.replace([np.inf], 0, inplace=True)

In [38]:
scaled_sum = scaled_pixels.sum()

In [39]:
raw_sum = corrected_pixels['counts'].sum()

In [40]:
scale = np.sqrt(scaled_sum/raw_sum)
scale

0.0005534767907410328

In [44]:
scaled_rowSums = corrected_rowSums * scale
scaled_rowSums.values

array([0.        , 0.        , 0.02988775, ..., 0.        , 0.        ,
       0.        ])

In [45]:
# Save scaled rowsums back into hdf
del f['/resolutions/{}/bins/test_weights'.format(res)]
f.create_dataset('/resolutions/{}/bins/test_weights'.format(res), data=scaled_rowSums)
f.close()

In [51]:
clr = cooler.Cooler("{}::resolutions/{}".format(cool, res))
clr.bins()[5000:5010]

,chrom,start,end,KR,VC,VC_SQRT,test_weights
5000,1,25000000,25005000,0.766987,0.765568,0.760951,0.689632
5001,1,25005000,25010000,0.802792,0.801334,0.778523,0.732803
5002,1,25010000,25015000,0.707450,0.699996,0.727633,0.680776
5003,1,25015000,25020000,0.816580,0.829436,0.792057,0.889437
5004,1,25020000,25025000,0.780722,0.791115,0.773543,0.812504
5005,1,25025000,25030000,0.780396,0.780045,0.768112,0.730036
5006,1,25030000,25035000,0.622555,0.585034,0.665204,0.627089
5007,1,25035000,25040000,0.685397,0.668488,0.711069,0.650889
5008,1,25040000,25045000,0.929117,0.951212,0.848210,0.896079
5009,1,25045000,25050000,0.601072,0.572260,0.657902,0.540747
